<a href="https://colab.research.google.com/github/chaehchaeh/MajorAI-DL/blob/main/221031DL6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

cancer=load_breast_cancer()
df=pd.DataFrame(cancer.data,columns=cancer.feature_names)
df['class']=cancer.target

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
data=torch.from_numpy(df.values).float()

x=data[:,:-1]
y=data[:,-1:]
print(x.shape,y.shape)

torch.Size([569, 30]) torch.Size([569, 1])


In [11]:
ratios=[.6,.2,.2]

train_cnt=int(data.size(0)*ratios[0])
valid_cnt=int(data.size(0)*ratios[1])
test_cnt=data.size(0)-train_cnt-valid_cnt
cnts=[train_cnt,valid_cnt,test_cnt]

print("Train %d/ Valid %d /Test %d samples."% (train_cnt,valid_cnt,test_cnt))
              

Train 341/ Valid 113 /Test 115 samples.


In [19]:
indices=torch.randperm(data.size(0))

x=torch.index_select(x, dim=0,index=indices)
y=torch.index_select(y, dim=0, index=indices)

x=x.split(cnts, dim=0)
y=y.split(cnts, dim=0)
for x_i,y_i in zip(x,y):
    print(x_i.size(),y_i.size())

TypeError: ignored

In [21]:
#데이터 정규화
scaler=StandardScaler()
scaler.fit(x[0].numpy())
x=[torch.from_numpy(scaler.transform(x[0].numpy())).float(),
   torch.from_numpy(scaler.transform(x[1].numpy())).float(),
   torch.from_numpy(scaler.transform(x[2].numpy())).float()]


In [24]:
#심층신경망 모델 구성 및 옵티마이저 설정
model=nn.Sequential(
    nn.Linear(x[0].size(-1),25),
    nn.LeakyReLU(),
    nn.Linear(25,20),
    nn.LeakyReLU(),
    nn.Linear(20,15),
    nn.LeakyReLU(),
    nn.Linear(15,10),
    nn.LeakyReLU(),
    nn.Linear(10,5),
    nn.LeakyReLU(),
    nn.Linear(5,y[0].size(-1)),
    nn.Sigmoid()
)

optimizer=optim.Adam(model.parameters())

In [26]:
#하이퍼 파라미터 설정
n_epochs=10000
batch_size=32
print_interval=10
early_stop=100
lowest_loss=np.inf
best_model=None
lowest_epoch=np.inf

In [27]:
#이진분류 심층신경망 구현

train_history, valid_history=[],[]

for i in range(n_epochs):
    indices=torch.randperm(x[0].size[0])
    x_=torch.index_select(x[0],dim=0, index=indices)
    y_=torch.index_select(y[0],dim=0, index=indices)

    x_=x_.split(batch_size, dim=0)
    y_=y_.split(batch_size, dim=0)

    train_loss,valid_loss=0,0
    y_hat=[]

    for x_i,y_i in zip(x_,y_):
        y_hat_i=model(x_i)
        loss=F.binary_cross_entropy(y_hat_i,y_i)
        
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        train_loss+=float(loss)
        
    train_loss=train_loss/len(x_)

    with torch.no_grad():
        x_=x[1].split(batch_size, dim=0)
        y_=y[1].split(batch_size, dim=0)

        valid_loss=0

        for x_i,y_i in zip(x_,y_):
            y_hat_i=model(x_i)
            loss=F.binary_cross_entropy(y_hat_i,y_i)

            valid_loss+= float(loss)

            y_hat+=[y_hat_i]

    valid_loss=valid_loss/len(x_)

    train_history+=[train_loss]


TypeError: ignored